In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
!pip install transformers
!pip install simpletransformers

In [3]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.9.0+cu102', '4.8.1', '0.10.3')

In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [6]:
data_file = pd.read_csv("data/trofi.csv")

In [7]:
data_file = data_file.rename(columns={'sentence': 'input_text'}) #abstract - sentence
#data_file['target_text'] = data[['arg1', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [8]:
papers = data_file[["input_text", "target_text"]]
papers.head()

,input_text,target_text
0,An Energy Department spokesman says the sulfur...,absorb
1,The yellow beta carotene pigment absorbs blue ...,absorb
2,"This time , the ground absorbed the shock wave...",absorb
3,'' Vitamins could be passed right out of the b...,absorb
4,"As Eliot wrote : '' In a warm haze , the sultr...",absorb


In [9]:
# Load test data - TroFi
import pandas as pd
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd

# Split to train, val and test
train_df, test_data = tts(data_file[["input_text", "target_text"]], random_state=42, test_size=0.1)
train_df, val = tts(train_df, random_state=42, test_size=test_data.shape[0])

In [10]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

eval_df = pd.read_csv("stockholm/xlm_code/trofi_xlm_subs/trofi_tp.csv")
eval_df = eval_df.rename(columns={'sentence': 'input_text'}) #abstract - sentence
eval_df['target_text'] = eval_df['verb'] #title - metaphorical words

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 6,
    "num_train_epochs": 3,
}

# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

INFO:filelock:Lock 139792516721744 acquired on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.8512cdf8592f538a7fd4b40eecaa096285410ec6494049568b3300922ab71165.lock


Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

INFO:filelock:Lock 139792516721744 released on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.8512cdf8592f538a7fd4b40eecaa096285410ec6494049568b3300922ab71165.lock
INFO:filelock:Lock 139792516721744 acquired on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

INFO:filelock:Lock 139792516721744 released on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock
INFO:filelock:Lock 139792384568528 acquired on /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock


...

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

INFO:filelock:Lock 139792384604688 released on /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
INFO:filelock:Lock 139792384529168 acquired on /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

INFO:filelock:Lock 139792384529168 released on /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


CPU times: user 17 s, sys: 3.52 s, total: 20.5 s
Wall time: 26 s


In [11]:
%%time

# Train the model
model.train_model(train_df)

# Evaluate the model
result = model.eval_model(eval_df)
print(result)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2989 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/499 [00:00<?, ?it/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/137 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.13188032771097843}


{'eval_loss': 0.13188032771097843}
CPU times: user 13min 52s, sys: 1min 29s, total: 15min 22s
Wall time: 15min 26s


In [13]:
for _ in range(65):

    random_idx = random.randint(0, len(eval_df)-1)

    abstract = eval_df.iloc[random_idx]['input_text']
    true_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained BART model
    predicted_title = model.predict([abstract])[0]

    print(f'True Title: {true_title}\n')
    print(f'Predicted Title: {predicted_title}\n')
    print(f'Abstract: {abstract}\n\n\n')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: eat

Predicted Title: eat

Abstract: Out - of - home entertainment -- such as films , sporting events , eating out , dancing , theater , shopping and amusement parks -- was a leisure - time activity for 43 % of respondents . -LRB- The survey included multiple responses . -RRB-/-R





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: sleep

Predicted Title: sleep

Abstract: Perhaps the Logan regulators have made themselves a bed they can not sleep in , suggests Mr. Smith : '' An ideology of radical egalitarianism that sees people as uniform blocks of mud is what gives you the idea that 200 scouts are better than two CEOs . '





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

True Title: roll

Predicted Title: roll

Abstract: Charity officials say they need trustees who are willing to roll up their sleeves and work for the organization and who will back up their commitment with donations 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: knock

Predicted Title: knock

Abstract: Seagram must decide by Feb. 9 whether to come up with a higher offer that would knock Grand Met out of the bidding war and frustrate its expansion strategy 





Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

True Title: cool

Predicted Title: cool

Abstract: A surprisingly strong October employment report , implying that the economy is n't cooling off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said 



